In [1]:
from __future__ import division # safety with double division
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
Opt = SolverFactory("gurobi")

# Clustering for Centroids
In this linear programming optimization, it will be optimizing for the Centroids location relative to the fixed assignments;

### M1:
This LP will fix the assignment of the Centroids and optimize for the Lowest possible distance of $C_i$ 

In [2]:
M1 = AbstractModel()
M1.name = "Clustering Centroid-Distance LP"

### M2
This LP will fix the Centroid Locations and redo the assignement. 

In [3]:
M2 = AbstractModel()
M2.name = "Clustering Assignment-LP"

## Parameters
Both LP's have same starting parameter that characterize the problem.
- **d**: number of dimensions
- **n**: number of points to cluster
- **k**: number of clusters to generate

In [4]:
M1.NumberOfDimensions = Param(within=NonNegativeIntegers)
M1.NumberOfPoints = Param(within=NonNegativeIntegers)
M1.NumberOfClusters = Param(within=NonNegativeIntegers)

In [5]:
M2.NumberOfDimensions = Param(within=NonNegativeIntegers)
M2.NumberOfPoints = Param(within=NonNegativeIntegers)
M2.NumberOfClusters = Param(within=NonNegativeIntegers)

## Set
- **Dimension Index (D)**: Set consisting of all possible possible dimensions an arbitrary point i.e. $\{x_1, x_2, x_3 ... x_d\}$
- **Points (P)**: Set consisting of all indexes for Points in the system. $\{p_1, p_2, p_3 ... p_n\}$
- **Cluster Index (C)**: Set consisting of possible ClusterIndex. $\{c_1, c_2, c_3 ... c_k\}$

In [6]:
M1.DimensionIndex = RangeSet(1,M1.NumberOfDimensions)
M1.PointsIndex = RangeSet(1,M1.NumberOfPoints)
M1.ClusterIndex = RangeSet(1,M1.NumberOfClusters)

In [7]:
M2.DimensionIndex = RangeSet(1,M2.NumberOfDimensions)
M2.PointsIndex = RangeSet(1,M2.NumberOfPoints)
M2.ClusterIndex = RangeSet(1,M2.NumberOfClusters)

## Inputs
- **Point**: $P_{i,d}$ where $i$ $\in$ PointsIndex and $j$ $\in$ DimensionIndex 

In [8]:
M1.Point = Param(M1.PointsIndex,M1.DimensionIndex, within=Reals)

In [9]:
M2.Point = Param(M2.PointsIndex,M2.DimensionIndex, within=Reals)

## Possible Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ dimensionalIndex 
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

### M1 Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ D
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

In [10]:
M1.Centroid=Var(M1.ClusterIndex, M1.DimensionIndex, within=Reals)
M1.Slack = Var(M1.PointsIndex,M1.ClusterIndex,M1.DimensionIndex, within=Reals)
M1.Z = Var(M1.PointsIndex,M1.ClusterIndex, within=NonNegativeReals)

### M2 Variables
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

In [11]:
M2.Assignment = Var(M2.PointsIndex, M2.ClusterIndex, within=Binary)
M2.Slack = Var(M2.PointsIndex,M2.ClusterIndex,M2.DimensionIndex, within=Reals)
M2.Z = Var(M2.PointsIndex,M2.ClusterIndex, within=NonNegativeReals)

### M1 Fixed Value

In [12]:
M1.Assignment=Param(M1.PointsIndex, M1.ClusterIndex, default=0,within=Binary, mutable=True)

### M2 Fixed Value

In [13]:
M2.Centroid=Param(M2.ClusterIndex, M2.DimensionIndex,default=0,within=Reals, mutable=True)

# Model M1
## Objective Function for M1
$$ \textbf{min}  \;\sum_{i\in P}\sum_{j\in C} Z $$

In [14]:
def ObjectiveFunction(M):
    return sum(\
               M.Z[i,j]\
               for i in M.PointsIndex \
               for j in M.ClusterIndex)
M1.Distance = Objective(rule=ObjectiveFunction, sense=minimize)
    

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [15]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack[i,j,d]
M1.Norm = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = DistanceConstraint)

### Constraint 2: Within Lower Bound
$$ -Z_{i,j} \leq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [16]:
def LowerBoundConstraint1(M,i,j,d):
    return -M.Z[i,j] <=M.Slack[i,j,d]
M1.LowerZBound = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = LowerBoundConstraint1)

### Constraint 3: With Upper Bound
$$ Z_{i,j} \geq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [17]:
def UpperBoundConstraint1(M,i,j,d):
    return M.Z[i,j] >=M.Slack[i,j,d]
M1.UpperZBound = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = UpperBoundConstraint1)

# Model M2
## Objective Function for M2
$$ \textbf{min}  \;\sum_{i\in P}\sum_{j\in C} Z $$

In [18]:
def ObjectiveFunction(M):
       return sum(\
               M.Z[i,j]\
               for i in M.PointsIndex \
               for j in M.ClusterIndex)
M2.Distance = Objective(rule=ObjectiveFunction, sense=minimize)

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [19]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack[i,j,d]
M2.Norm = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = DistanceConstraint)

### Constraint 2: Within Lower Bound
$$ -Z_{i,j} \leq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [20]:
def LowerBoundConstraint2(M,i,j,d):
    return -M.Z[i,j]<=M.Slack[i,j,d]
M2.LowerZBound = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = LowerBoundConstraint2)

### Constraint 3: With Upper Bound
$$ Z_{i,j} \geq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [21]:
def UpperBoundConstraint2(M,i,j,d):
    return M.Z[i,j] >=M.Slack[i,j,d]
M2.UpperZBound = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = UpperBoundConstraint2)

### Constraint 4: Non-Empty Cluster
$$ 1\leq \sum_{i \in P} A_{i,j} \qquad \forall j \in C $$

In [22]:
def NonEmptyCluster(M, j):
    return 1<=sum(M.Assignment[i,j] for i in M.PointsIndex)
M2.NonEmptyBalance = Constraint(M2.ClusterIndex, rule = NonEmptyCluster)

### Constraint 5: Singular Assignment 
$$ 1 = \sum_j A_{i,j} \qquad \forall i \in P$$

In [23]:
def SingularAssignment(M, i):
    return 1==sum(M.Assignment[i,j] for j in M.ClusterIndex)
M2.SingularAssignmentBalance = Constraint(M2.PointsIndex, rule = SingularAssignment)

## Create Problem and Solver Instance

In [42]:
folder =  "../Data/2Big/"
dat_location ="200Points"
dat_file = folder+dat_location+".dat"
original_instance1 = M1.create_instance(dat_file)
original_instance2 = M2.create_instance(dat_file)
clusters = original_instance1.NumberOfClusters.value
points = original_instance1.NumberOfPoints.value
dimensions = original_instance1.NumberOfDimensions.value

In [43]:
cluster_column = ["iteration", "index"]+["x_"+str(d) for d in range(1,dimensions+1)]
centroid_DF = pd.DataFrame(columns=cluster_column)

In [44]:
assignment_column  = ["iteration","P_i","C_j"]
assignment_DF = pd.DataFrame(columns=assignment_column)

In [45]:
def seeding_A(instance):
    clusters = instance.NumberOfClusters.value
    points = instance.NumberOfPoints.value
    for x in range(1,points+1):
        instance.Assignment[x,((x-1)%clusters)+1]=1

In [46]:
past_instance1 = None
current_instance1 = original_instance1.clone()
past_instance2 = None
current_instance2 = original_instance2.clone()
seeding_A(current_instance1)
count=0

while(True):
    def end_condition_1(past_instance1, current_instance1):
        if(past_instance1 is not None and past_instance2 is not None):
            if value(past_instance1.Distance) == value(current_instance1.Distance):
                for j in range(1,clusters+1):
                    for d in range(1, dimensions+1):
                        if past_instance1.Centroid[j,d] != current_instance1.Centroid[j,d]:
                            return False
                return True
        return False
    
    def end_condition_2(past_instance2, current_instance2):
        if(past_instance2 is not None):
            if value(past_instance2.Distance) == value(current_instance2.Distance):
                for i in range(1,points+1):
                    for j in range(1, clusters+1):
                        if past_instance2.Assignment[i,j] != current_instance2.Assignment[i,j]:
                            return False
                return True
        return False

    Soln1 = Opt.solve(current_instance1)
    current_instance1.solutions.load_from(Soln1)

    # Print the output
    print("Current_instance 1: ", count)
    print("Termination Condition was "+str(Soln1.Solver.Termination_condition))
    display(current_instance1)
    
    # Record Locations of Stuff
    for i in range(1,points+1):
        for j in range(1,clusters+1):
            if(current_instance1.Assignment[i,j] == 1):
                arr = [count, i, j]
                assignment_DF.loc[len(assignment_DF)] = arr
    
    for j in range(1,clusters+1):
        arr = [count, j]
        for d  in range(1,dimensions+1):
            arr.append(value(current_instance1.Centroid[j,d]))
        centroid_DF.loc[len(centroid_DF)] = arr



    if end_condition_1(past_instance1,current_instance1):
        break;
    past_instance1 = current_instance1
    current_instance2 = original_instance2.clone()
    
    #Hand Off
    for j in range(1,clusters+1):
        for d in range(1,dimensions+1):
            current_instance2.Centroid[j,d]=current_instance1.Centroid[j,d]
    count+=1
    
    #Solve M2
    Soln2 = Opt.solve(current_instance2)
    current_instance2.solutions.load_from(Soln2)
    
    print("Current_instance 2: ", count)
    print("Termination Condition was "+str(Soln2.Solver.Termination_condition))
    
    display(current_instance2)
    
    ## Assignment Recording
    for i in range(1,points+1):
        for j in range(1,clusters+1):
            if(current_instance2.Assignment[i,j] == 1):
                arr = [count, i, j]
                assignment_DF.loc[len(assignment_DF)] = arr
    
    ## Centroid Location
    for j in range(1,clusters+1):
        arr = [count, j]
        for d  in range(1,dimensions+1):
            arr.append(value(current_instance2.Centroid[j,d]))
        centroid_DF.loc[len(centroid_DF)] = arr

    
    if end_condition_2(past_instance2, current_instance2):
        break;
    
    past_instance2 = current_instance2
    current_instance1 = original_instance1.clone()
    
    for i in range(1,points+1):
            for j in range(1,clusters+1):
                current_instance1.Assignment[i,j]=current_instance2.Assignment[i,j]
    
    count+=1

Current_instance 1:  0
Termination Condition was optimal
Model Clustering Centroid-Distance LP

  Variables:
    Centroid : Size=4, Index=Centroid_index
        Key    : Lower : Value          : Upper : Fixed : Stale : Domain
        (1, 1) :  None : -1.26285161788 :  None : False : False :  Reals
        (1, 2) :  None :  1.14618510736 :  None : False : False :  Reals
        (2, 1) :  None : -1.43964263687 :  None : False : False :  Reals
        (2, 2) :  None :   1.4562570645 :  None : False : False :  Reals
    Slack : Size=800, Index=Slack_index
        Key         : Lower : Value            : Upper : Fixed : Stale : Domain
          (1, 1, 1) :  None :   -2.83074999659 :  None : False : False :  Reals
          (1, 1, 2) :  None :    3.50021822398 :  None : False : False :  Reals
          (1, 2, 1) :  None :              0.0 :  None : False : False :  Reals
          (1, 2, 2) :  None :              0.0 :  None : False : False :  Reals
          (2, 1, 1) :  None :             

        (100, 2, 1) :  None :       -1.428282480643 :   0.0
        (100, 2, 2) :  None :                   0.0 :   0.0
        (101, 1, 1) :  None :                   0.0 :   0.0
        (101, 1, 2) :  None :       -2.216553731668 :   0.0
        (101, 2, 1) :  None :                   0.0 :   0.0
        (101, 2, 2) :  None :                   0.0 :   0.0
        (102, 1, 1) :  None :                   0.0 :   0.0
        (102, 1, 2) :  None :                   0.0 :   0.0
        (102, 2, 1) :  None :   -0.7198490870440001 :   0.0
        (102, 2, 2) :  None :        -2.35559600426 :   0.0
        (103, 1, 1) :  None :        -2.57483323586 :   0.0
        (103, 1, 2) :  None :        -0.56518235136 :   0.0
        (103, 2, 1) :  None :                   0.0 :   0.0
        (103, 2, 2) :  None :                   0.0 :   0.0
        (104, 1, 1) :  None :                   0.0 :   0.0
        (104, 1, 2) :  None :                   0.0 :   0.0
        (104, 2, 1) :  None :         -2

Current_instance 2:  1
Termination Condition was optimal
Model Clustering Assignment-LP

  Variables:
    Assignment : Size=400, Index=Assignment_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :   1.0 :     1 : False : False : Binary
          (1, 2) :     0 :   0.0 :     1 : False : False : Binary
          (2, 1) :     0 :   1.0 :     1 : False : False : Binary
          (2, 2) :     0 :   0.0 :     1 : False : False : Binary
          (3, 1) :     0 :   1.0 :     1 : False : False : Binary
          (3, 2) :     0 :   0.0 :     1 : False : False : Binary
          (4, 1) :     0 :   1.0 :     1 : False : False : Binary
          (4, 2) :     0 :   0.0 :     1 : False : False : Binary
          (5, 1) :     0 :   1.0 :     1 : False : False : Binary
          (5, 2) :     0 :   0.0 :     1 : False : False : Binary
          (6, 1) :     0 :   1.0 :     1 : False : False : Binary
          (6, 2) :     0 :   0.0 :     1 : False : False

         (38, 1, 1) :  None :    -2.7416322191800004 :   0.0
         (38, 1, 2) :  None :          -8.8573400188 :   0.0
         (38, 2, 1) :  None :                    0.0 :   0.0
         (38, 2, 2) :  None :                    0.0 :   0.0
         (39, 1, 1) :  None :    -2.3677144868500006 :   0.0
         (39, 1, 2) :  None :         -8.27911673556 :   0.0
         (39, 2, 1) :  None :                    0.0 :   0.0
         (39, 2, 2) :  None :                    0.0 :   0.0
         (40, 1, 1) :  None :        -2.184310765711 :   0.0
         (40, 1, 2) :  None :         -6.11122955748 :   0.0
         (40, 2, 1) :  None :                    0.0 :   0.0
         (40, 2, 2) :  None :                    0.0 :   0.0
         (41, 1, 1) :  None :        -1.087083417036 :   0.0
         (41, 1, 2) :  None :        -1.176709385982 :   0.0
         (41, 2, 1) :  None :                    0.0 :   0.0
         (41, 2, 2) :  None :                    0.0 :   0.0
         (42, 1, 1) :  N

Current_instance 1:  2
Termination Condition was optimal
Model Clustering Centroid-Distance LP

  Variables:
    Centroid : Size=4, Index=Centroid_index
        Key    : Lower : Value           : Upper : Fixed : Stale : Domain
        (1, 1) :  None : -0.470743253974 :  None : False : False :  Reals
        (1, 2) :  None :  -0.25581767529 :  None : False : False :  Reals
        (2, 1) :  None :  -1.79940175059 :  None : False : False :  Reals
        (2, 2) :  None :   2.18034104313 :  None : False : False :  Reals
    Slack : Size=800, Index=Slack_index
        Key         : Lower : Value             : Upper : Fixed : Stale : Domain
          (1, 1, 1) :  None :    -2.03864163269 :  None : False : False :  Reals
          (1, 1, 2) :  None :     2.09821544133 :  None : False : False :  Reals
          (1, 2, 1) :  None :               0.0 :  None : False : False :  Reals
          (1, 2, 2) :  None :               0.0 :  None : False : False :  Reals
          (2, 1, 1) :  None :   

...

Current_instance 1:  10
Termination Condition was optimal
Model Clustering Centroid-Distance LP

  Variables:
    Centroid : Size=4, Index=Centroid_index
        Key    : Lower : Value          : Upper : Fixed : Stale : Domain
        (1, 1) :  None :  1.06109314628 :  None : False : False :  Reals
        (1, 2) :  None :  -2.4002355423 :  None : False : False :  Reals
        (2, 1) :  None : -1.41306779295 :  None : False : False :  Reals
        (2, 2) :  None :   1.7164363015 :  None : False : False :  Reals
    Slack : Size=800, Index=Slack_index
        Key         : Lower : Value             : Upper : Fixed : Stale : Domain
          (1, 1, 1) :  None :   -0.506805232437 :  None : False : False :  Reals
          (1, 1, 2) :  None :  -0.0462024256862 :  None : False : False :  Reals
          (1, 2, 1) :  None :               0.0 :  None : False : False :  Reals
          (1, 2, 2) :  None :               0.0 :  None : False : False :  Reals
          (2, 1, 1) :  None :    0.8

        (170, 1, 2) :  None :                   0.0 :   0.0
        (170, 2, 1) :  None :       -1.255642732136 :   0.0
        (170, 2, 2) :  None :                   0.0 :   0.0
        (171, 1, 1) :  None :                   0.0 :   0.0
        (171, 1, 2) :  None :                   0.0 :   0.0
        (171, 2, 1) :  None :        -3.99676035604 :   0.0
        (171, 2, 2) :  None :       -2.384538950162 :   0.0
        (172, 1, 1) :  None :                   0.0 :   0.0
        (172, 1, 2) :  None :                   0.0 :   0.0
        (172, 2, 1) :  None :       -1.230691195254 :   0.0
        (172, 2, 2) :  None :                   0.0 :   0.0
        (173, 1, 1) :  None :                   0.0 :   0.0
        (173, 1, 2) :  None :                   0.0 :   0.0
        (173, 2, 1) :  None :         -2.1881200187 :   0.0
        (173, 2, 2) :  None :  -0.03146132150999992 :   0.0
        (174, 1, 1) :  None :                   0.0 :   0.0
        (174, 1, 2) :  None :           

Current_instance 2:  11
Termination Condition was optimal
Model Clustering Assignment-LP

  Variables:
    Assignment : Size=400, Index=Assignment_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :   1.0 :     1 : False : False : Binary
          (1, 2) :     0 :   0.0 :     1 : False : False : Binary
          (2, 1) :     0 :   1.0 :     1 : False : False : Binary
          (2, 2) :     0 :   0.0 :     1 : False : False : Binary
          (3, 1) :     0 :   1.0 :     1 : False : False : Binary
          (3, 2) :     0 :   0.0 :     1 : False : False : Binary
          (4, 1) :     0 :   1.0 :     1 : False : False : Binary
          (4, 2) :     0 :   0.0 :     1 : False : False : Binary
          (5, 1) :     0 :   1.0 :     1 : False : False : Binary
          (5, 2) :     0 :   0.0 :     1 : False : False : Binary
          (6, 1) :     0 :   1.0 :     1 : False : False : Binary
          (6, 2) :     0 :   0.0 :     1 : False : Fals

        Key         : Lower : Body                   : Upper
          (1, 1, 1) :  None :                    0.0 :   0.0
          (1, 1, 2) :  None :       -0.4606028067516 :   0.0
          (1, 2, 1) :  None :                    0.0 :   0.0
          (1, 2, 2) :  None :                    0.0 :   0.0
          (2, 1, 1) :  None :         -1.63386048099 :   0.0
          (2, 1, 2) :  None :   -0.06347260735999993 :   0.0
          (2, 2, 1) :  None :                    0.0 :   0.0
          (2, 2, 2) :  None :                    0.0 :   0.0
          (3, 1, 1) :  None :        -1.491149042249 :   0.0
          (3, 1, 2) :  None :        -1.918087823796 :   0.0
          (3, 2, 1) :  None :                    0.0 :   0.0
          (3, 2, 2) :  None :                    0.0 :   0.0
          (4, 1, 1) :  None :   -0.07574279752399998 :   0.0
          (4, 1, 2) :  None :                    0.0 :   0.0
          (4, 2, 1) :  None :                    0.0 :   0.0
          (4, 2, 2) :  N

         71 :   1.0 :  1.0 :   1.0
         72 :   1.0 :  1.0 :   1.0
         73 :   1.0 :  1.0 :   1.0
         74 :   1.0 :  1.0 :   1.0
         75 :   1.0 :  1.0 :   1.0
         76 :   1.0 :  1.0 :   1.0
         77 :   1.0 :  1.0 :   1.0
         78 :   1.0 :  1.0 :   1.0
         79 :   1.0 :  1.0 :   1.0
         80 :   1.0 :  1.0 :   1.0
         81 :   1.0 :  1.0 :   1.0
         82 :   1.0 :  1.0 :   1.0
         83 :   1.0 :  1.0 :   1.0
         84 :   1.0 :  1.0 :   1.0
         85 :   1.0 :  1.0 :   1.0
         86 :   1.0 :  1.0 :   1.0
         87 :   1.0 :  1.0 :   1.0
         88 :   1.0 :  1.0 :   1.0
         89 :   1.0 :  1.0 :   1.0
         90 :   1.0 :  1.0 :   1.0
         91 :   1.0 :  1.0 :   1.0
         92 :   1.0 :  1.0 :   1.0
         93 :   1.0 :  1.0 :   1.0
         94 :   1.0 :  1.0 :   1.0
         95 :   1.0 :  1.0 :   1.0
         96 :   1.0 :  1.0 :   1.0
         97 :   1.0 :  1.0 :   1.0
         98 :   1.0 :  1.0 :   1.0
         99 :   1.0 

In [47]:
assignment_DF.to_csv(folder+dat_location+"_Assignment_INF_NORM.csv")

In [48]:
centroid_DF.to_csv(folder+dat_location+"_Centroid_INF_NORM.csv")

In [49]:
count

11